# Welcome to VARIANT in Databricks SQL

**What is VARIANT?**

In Delta Lake, VARIANT is a data type that allows for efficient storage and query of semi-structured data formats such as JSON and Avro within Delta tables and soon in Apache Iceberg in V3. This is particularly useful for workloads where data structures may change over time or are not fully known in advance, such as event streaming, IoT, and complex nested datasets.

**Why does it matter?**

* JSON is everywhere: logs, events, API data, and more.
* Traditional STRING storage is slow and error-prone for JSON.
* VARIANT makes it easy, fast, and reliable to work with complex data.

**What will you learn?**

* How to store JSON as STRING vs VARIANT
* How to extract, filter, and analyze JSON fields
* How VARIANT makes analytics easier and faster

Let's get started!

In [0]:
%sql
-- Create a table with both STRING and VARIANT columns for JSON
CREATE TABLE IF NOT EXISTS tmp.variant_vs_string_demo (
  id INT,
  json_string STRING,
  json_variant VARIANT
);

-- Insert sample data
INSERT INTO tmp.variant_vs_string_demo
SELECT * FROM (
  SELECT 1 AS id,
    '{"user": {"id": 1, "name": "Alice", "age": 30}, "event": {"type": "login", "platform": "web"}}' AS json_string,
    parse_json('{"user": {"id": 1, "name": "Alice", "age": 30}, "event": {"type": "login", "platform": "web"}}') AS json_variant
  UNION ALL
  SELECT 2,
    '{"user": {"id": 2, "name": "Bob", "age": 25}, "event": {"type": "purchase", "platform": "mobile", "amount": 99.99}}',
    parse_json('{"user": {"id": 2, "name": "Bob", "age": 25}, "event": {"type": "purchase", "platform": "mobile", "amount": 99.99}}')
  UNION ALL
  SELECT 3,
    '{"user": {"id": 3, "name": "Carol", "age": 40}, "event": {"type": "login", "platform": "mobile"}}',
    parse_json('{"user": {"id": 3, "name": "Carol", "age": 40}, "event": {"type": "login", "platform": "mobile"}}')
) t
WHERE NOT EXISTS (SELECT 1 FROM tmp.variant_vs_string_demo);

-- Show sample data
SELECT * FROM tmp.variant_vs_string_demo;

## Storing JSON: STRING vs VARIANT

Below, we create a table with two ways to store JSON:
* As a plain STRING (hard to query)
* As a VARIANT (easy to query)

We'll use the same sample data for both columns.

In [0]:
%sql
-- Extract user name and event type from JSON string using JSON functions
SELECT 
  id,
  get_json_object(json_string, '$.user.name') AS user_name_from_string,
  get_json_object(json_string, '$.event.type') AS event_type_from_string
FROM tmp.variant_vs_string_demo;


In [0]:
%sql
SELECT 
  id,
  json_variant:user.name::string AS user_name_from_variant,
  json_variant:event.type::string AS event_type_from_variant
FROM tmp.variant_vs_string_demo;

**Why is VARIANT faster and better than STRING for JSON?**

* **Native Parsing:** VARIANT stores JSON in a parsed, binary format, so queries do not need to repeatedly parse text—unlike STRING, which requires parsing on every query.
* **Efficient Indexing:** Databricks can index and optimize queries on VARIANT fields, enabling faster lookups and filtering.
* **Direct Path Access:** You can access nested fields directly (e.g., `json_variant:user.name`), avoiding slow and error-prone string parsing functions.
* **Schema Evolution:** VARIANT supports flexible, evolving schemas without breaking queries or requiring table changes.
* **Better Compression:** Binary storage of VARIANT is more compact than raw JSON strings, improving I/O and scan speed.
* **Type Safety:** Queries on VARIANT can enforce types (e.g., `::string`, `::int`), reducing runtime errors and improving performance.

In [0]:
%sql
-- Count number of login events using VARIANT (fast, direct access)
SELECT COUNT(*) AS login_events_variant
FROM tmp.variant_vs_string_demo
WHERE json_variant:event.type::string = 'login';

-- Count number of login events using STRING (requires parsing every row)
SELECT COUNT(*) AS login_events_string
FROM tmp.variant_vs_string_demo
WHERE get_json_object(json_string, '$.event.type') = 'login';

**Schema Discovery with VARIANT**

VARIANT makes it easy to discover and explore the structure of semi-structured JSON data. You can use built-in functions to infer the schema of your data, which is much harder with plain strings.

In [0]:
%sql


-- Infer schema from a VARIANT column
SELECT schema_of_variant(json_variant) AS inferred_schema_from_variant
FROM tmp.variant_vs_string_demo
LIMIT 1;

In [0]:
%sql
-- Extract a nested field directly
SELECT id, json_variant:user.name::string AS user_name FROM tmp.variant_vs_string_demo;


In [0]:
%sql
SELECT id, typeof(json_variant) AS variant_type FROM main.tmp.variant_vs_string_demo;

# Key Takeaways: Why Use VARIANT?

* VARIANT is designed for modern, semi-structured data like JSON.
* It makes querying, filtering, and analyzing data much easier and faster.
* You get type safety, schema discovery, and powerful built-in functions.
* Stop struggling with strings—use VARIANT for your next analytics project!

In [0]:
%sql
SELECT 
  id,
  variant_get(json_variant, "user.name", 'STRING') AS user_name
FROM tmp.variant_vs_string_demo;